# In-Class Activity: Understanding BLAST through Python

This live coding activity demonstrates the balance between speed and accuracy in BLAST, highlighting how efficient heuristics can narrow down the search space before applying more computationally intensive algorithms.

Objectives:

Implement the core steps of BLAST, including:

1. Building a lookup table of k-mers (seeding).
2. Screening a synthetic FASTA database.
3. Refining promising regions with a Smith–Waterman local alignment.

## Step 1: Fetch and Parse the FASTA File

BLAST (Basic Local Alignment Search Tool) is used to find regions of similarity between biological sequences.
The first step in running a BLAST search is having a properly formatted query and database.
To understand how BLAST operates, we will start by retrieving and parsing a synthetic FASTA database.

### Step 1A: Fetching the FASTA File

BLAST requires sequences to be in FASTA format. We will retrieve a synthetic FASTA file from a remote source and store its contents as a string.

In [1]:
import requests

# URL to the synthetic FASTA database hosted on GitHub
FASTA_URL = (
    "https://github.com/oasci/pitt-biosc1540-2025s/raw/refs/heads/main/content/data/fasta/synthetic.fasta"
)

# Attempt to fetch the FASTA file; raise an exception if not successful.
response = requests.get(FASTA_URL)
if response.status_code == 200:
    fasta_text = response.text
else:
    raise Exception(f"Failed to fetch file. Status code: {response.status_code}")


Discussion Points

- Why is it necessary to fetch sequence data from a database before running BLAST?
- What issues could arise if the request fails? How can we handle them?
- What happens if the file is large? Would downloading the entire file into memory be a good idea? 

### Step 1C: Parsing the FASTA File

To work with sequence data programmatically, we need to convert the text-based FASTA file into a structured format.

The `read_fasta()` function reads a FASTA file and returns a dictionary where:

- Keys are sequence headers (without `>`).
- Values are concatenated sequences (handling multi-line sequences).  

In [2]:

def read_fasta(data: str) -> dict:
    """
    Parse a FASTA-formatted string and return a dictionary mapping sequence headers to sequences.
    
    Parameters:
        data: The FASTA file content as a single string.
    
    Returns:
        A dictionary where keys are headers (without the '>') and values are the concatenated sequences.
    """
    fasta_dict = {}
    header = None
    sequence_lines = []
    
    for line in data.splitlines():
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        if line.startswith(">"):
            # Save the previous record if it exists
            if header is not None:
                fasta_dict[header] = "".join(sequence_lines)
            header = line[1:]  # Remove '>' from header
            sequence_lines = []  # Reset sequence accumulator
        else:
            sequence_lines.append(line)
    
    # Save the last parsed record
    if header is not None:
        fasta_dict[header] = "".join(sequence_lines)
    
    return fasta_dict

### Step 1D: Running the Parser and Previewing Sequences

Now, we use `read_fasta()` to convert the raw FASTA text into a dictionary.

In [3]:
# Parse the FASTA content into a dictionary of sequences.
sequences = read_fasta(fasta_text)

# Display a preview of the first three sequences (first 50 bases)
print("=== FASTA File Preview ===")
for header, seq in list(sequences.items())[:3]:
    print(f">{header}\n{seq[:50]}...\n")

=== FASTA File Preview ===
>synthetic_seq1
GCATGTAGGCGCTGGACTCGCTAGTAGTTTTGGGGCTGGAGACCGGAAAA...

>synthetic_seq2
GGCGATAAGTTAAATTGTGTCAAGGGATGTCTTCGGAGTTCGAGCAACTG...

>synthetic_seq3
TTCGGAGCGTCCACCGCCTGTCCAAATTTCCATTGTAATGTTGTTGTTAA...



Discussion Points

1. Why do we check for `>` when parsing?
2. What happens if a sequence spans multiple lines?
3. Why do we use a dictionary?
4. What potential problems could occur with this function?

## Step 2: Build a Lookup Table for k-mers

Before performing full sequence alignment, BLAST first identifies short word (k-mer) matches between the query and database sequences.
This reduces computational complexity and speeds up the search process.

### Step 2A: What Are k-mers?

A k-mer is a short substring of length `k` found within a sequence.
For example, if `k = 5`, the sequence `"ATGCTAGC"` contains the k-mers:

```
"ATGCT", "TGCTA", "GCTAG", "CTAGC"
```

BLAST uses k-mers as seeds to quickly locate potential matches before performing detailed alignments.

Discussion Points

- Why does BLAST search for short k-mers first instead of performing full sequence alignment immediately?
- What happens if a k-mer does not exist in the database?
- How would repeats in a sequence affect k-mer lookup?

### Step 2B: Understanding the Impact of `word_size`

The word size (`k`) influences BLAST's sensitivity and speed:

| Word Size (`k`) | Effect on BLAST |
|-----------------|----------------|
| Smaller `k` | More sensitive but slower (more potential matches to evaluate). |
| Larger `k`  | Faster but may miss weaker or shorter matches. |

Discussion Points

- How does increasing `word_size` speed up the search?
- Why would decreasing `word_size` help in detecting more distant homologs?
- In what cases would a very small `k` cause too many false positives?  

### Step 2C: Implementing a k-mer Lookup Table

We will create a lookup table that maps each k-mer in a query sequence to the positions where it appears.

In [4]:
from collections import defaultdict

def build_lookup(sequence: str, word_size: int) -> dict:
    """
    Build a lookup table mapping each k-mer to a list of starting positions in the
    given sequence.
    
    Parameters:
        sequence: The sequence from which to extract k-mers.
        word_size: The length of each k-mer.
        
    Returns:
        A dictionary where keys are k-mers and values are lists of indices indicating
            where the k-mer occurs.
    """
    lookup = defaultdict(list)
    for i in range(len(sequence) - word_size + 1):
        kmer = sequence[i:i+word_size]
        lookup[kmer].append(i)
    return dict(lookup)

### Step 2D: Running the Lookup Table Code

Now, we apply the function to a sample query sequence:

In [5]:
# Define an example query sequence and the k-mer (word) size.
QUERY = "ATGCTAGCTAGCTACGATCGATCGATCTAGCTAGCTAGCTACGATCGATCG"
word_size = 5

# Build the lookup table for the query sequence.
lookup_table = build_lookup(QUERY, word_size)

# Display a few k-mer entries from the lookup table.
print("=== Lookup Table Preview ===")
for kmer, positions in list(lookup_table.items())[:5]:
    print(f"{kmer}: {positions}")

=== Lookup Table Preview ===
ATGCT: [0]
TGCTA: [1]
GCTAG: [2, 6, 29, 33]
CTAGC: [3, 7, 26, 30, 34]
TAGCT: [4, 8, 27, 31, 35]


Each k-mer is mapped to the positions where it appears in the sequence.

Discussion Points

- Why do some k-mers appear more than once?
- What happens if the sequence contains ambiguous bases (`N`)?
- How large would the lookup table be for a genome-scale query?

## Step 3: Find k-mer Matches in a Target Sequence

Instead of aligning an entire query sequence against every database sequence, BLAST first finds exact k-mer matches between the query and target sequences.
These seed matches allow BLAST to focus computational effort on only relevant regions, skipping large portions of non-matching sequences.

### Step 3A: Why Restrict the Search to k-mer Matches?

BLAST does not attempt a full sequence alignment initially.
Instead, it:

1. Finds k-mer matches between the query and target sequences.
2. Extends these matches into longer alignments if they meet a score threshold.
3. Skips any target sequences that have no k-mer matches.

This approach saves significant computation time, especially when searching large databases.

Discussion Points

- Why is it computationally expensive to align a query to every sequence in a database?
- How does searching only in matching regions reduce this cost?
- What happens if no k-mers from the query exist in the target sequence?

### Step 3B: Implementing k-mer Matching

We will now implement a function to find k-mer matches between a query and a target sequence.
This function compares the k-mers in a given target sequence against the lookup table we built earlier.

In [6]:
def find_kmer_matches(lookup_table: dict, target_sequence: str, word_size: int) -> dict:
    """
    Search for k-mer matches between the lookup table (from the query) and a target sequence.
    
    Parameters:
        lookup_table: The k-mer lookup table for the query sequence.
        target_sequence: The sequence in which to search for matching k-mers.
        word_size: The length of k-mers.
        
    Returns:
        dict: A dictionary mapping k-mers to a list of tuples. Each tuple contains:
              (position in target_sequence, corresponding position in query sequence).
    """
    matches = defaultdict(list)
    for i in range(len(target_sequence) - word_size + 1):
        kmer = target_sequence[i:i+word_size]
        if kmer in lookup_table:
            # For each occurrence of the k-mer in the query, record the match.
            for query_index in lookup_table[kmer]:
                matches[kmer].append((i, query_index))
    return dict(matches)

### Step 3C: Running the k-mer Matching Code

Now, we define a target sequence and use the function to find matching k-mers.

In [7]:
# Define an example target sequence (simulating a hit from a synthetic database).
TARGET = "TACGTAGCTAGCTGATCGATCGTAGCTAGCTAGCTAGCTGATCGATCGATCG"

# Find matching k-mers between the query and the target.
matches = find_kmer_matches(lookup_table, TARGET, word_size)

# Display found k-mer matches.
print("\n=== k-mer Matches in Target Sequence ===")
for kmer, positions in matches.items():
    print(f"{kmer}: {positions}")


=== k-mer Matches in Target Sequence ===
TAGCT: [(4, 4), (4, 8), (4, 27), (4, 31), (4, 35), (8, 4), (8, 8), (8, 27), (8, 31), (8, 35), (22, 4), (22, 8), (22, 27), (22, 31), (22, 35), (26, 4), (26, 8), (26, 27), (26, 31), (26, 35), (30, 4), (30, 8), (30, 27), (30, 31), (30, 35), (34, 4), (34, 8), (34, 27), (34, 31), (34, 35)]
AGCTA: [(5, 5), (5, 9), (5, 28), (5, 32), (5, 36), (23, 5), (23, 9), (23, 28), (23, 32), (23, 36), (27, 5), (27, 9), (27, 28), (27, 32), (27, 36), (31, 5), (31, 9), (31, 28), (31, 32), (31, 36)]
GCTAG: [(6, 2), (6, 6), (6, 29), (6, 33), (24, 2), (24, 6), (24, 29), (24, 33), (28, 2), (28, 6), (28, 29), (28, 33), (32, 2), (32, 6), (32, 29), (32, 33)]
CTAGC: [(7, 3), (7, 7), (7, 26), (7, 30), (7, 34), (25, 3), (25, 7), (25, 26), (25, 30), (25, 34), (29, 3), (29, 7), (29, 26), (29, 30), (29, 34), (33, 3), (33, 7), (33, 26), (33, 30), (33, 34)]
GATCG: [(13, 15), (13, 19), (13, 42), (13, 46), (17, 15), (17, 19), (17, 42), (17, 46), (39, 15), (39, 19), (39, 42), (39, 46)

Each k-mer is listed along with the positions where it appears:

- The first number in each tuple is the position in the target sequence.
- The second number is the position in the query sequence.

For example,  `TAGCT: [(5, 4), (10, 9), (25, 24)]`  means that the k-mer `"TAGCT"` appears:

- At position 5 in the target sequence (corresponding to position 4 in the query).
- Again at position 10 in the target sequence (9 in query).


### Step 3E: Key Takeaways

1. Why does this method improve efficiency?
2. What happens if a k-mer does not exist in the target sequence?
3. What if the target sequence contains errors or mutations?
4. How could we modify the function to allow approximate matches?


## Step 4: Refining Matches with Smith–Waterman Alignment

After identifying promising regions through k-mer matching, BLAST performs Smith–Waterman alignment to optimize the match score.
Unlike global alignment (Needleman–Wunsch), Smith–Waterman performs local alignment, meaning:

- It finds only the best-matching segment between two sequences.
- It allows alignments to start and end at different positions rather than forcing alignment of the full sequences.
- It assigns penalties for mismatches and gaps while rewarding matches.  

### Step 4A: Why Use Smith–Waterman?

BLAST’s goal is to find high-scoring local alignments, not necessarily to align entire sequences.
Smith–Waterman provides several advantages:

- Focuses only on relevant regions instead of aligning full sequences.
- Handles insertions, deletions, and substitutions using penalties.
- Can detect conserved motifs even in otherwise unrelated sequences.  

Discussion Points:

- How does Smith–Waterman differ from global alignment?
- Why does the algorithm set negative scores to zero instead of allowing negative scores to accumulate?
- What biological scenarios require local vs. global alignment?


### Step 4B: Implementing Smith–Waterman Alignment

We will reuse our Smith–Waterman alignment to refine the regions detected by k-mer matching.

In [8]:
def smith_waterman(seq1: str, seq2: str, match: int = 2, mismatch: int = -1, gap: int = -2) -> tuple[list[str], int]:
    """
    Perform local alignment of two sequences using the Smith-Waterman algorithm.
    
    Parameters:
        seq1: The first sequence.
        seq2: The second sequence.
        match: Score awarded for a match.
        mismatch: Penalty for a mismatch.
        gap: Penalty for a gap.
    
    Returns:
        A tuple (alignments, max_score) where:
            - alignments is a list of tuples (aligned_seq1, aligned_seq2) for all optimal local alignments.
            - max_score is the maximum alignment score.
    """
    m, n = len(seq1), len(seq2)
    score_matrix = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    
    max_score = 0
    max_positions = []
    
    # Fill in the score matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            diag = score_matrix[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            cell_score = max(0, diag, up, left)
            score_matrix[i][j] = cell_score

            if cell_score > max_score:
                max_score = cell_score
                max_positions = [(i, j)]
            elif cell_score == max_score and cell_score > 0:
                max_positions.append((i, j))
    
    def traceback(i: int, j: int):
        alignments = []
        
        # Base case: if we've reached a zero score or the edge of the matrix
        if score_matrix[i][j] == 0 or (i == 0 and j == 0):
            return [("", "")]
            
        current_score = score_matrix[i][j]
        
        # Check diagonal move
        if i > 0 and j > 0:
            diag_val = score_matrix[i - 1][j - 1]
            expected = diag_val + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            if current_score == expected:
                for a1, a2 in traceback(i - 1, j - 1):
                    alignments.append((a1 + seq1[i - 1], a2 + seq2[j - 1]))
        
        # Check up move
        if i > 0:
            up_val = score_matrix[i - 1][j]
            if current_score == up_val + gap:
                for a1, a2 in traceback(i - 1, j):
                    alignments.append((a1 + seq1[i - 1], a2 + "-"))
        
        # Check left move
        if j > 0:
            left_val = score_matrix[i][j - 1]
            if current_score == left_val + gap:
                for a1, a2 in traceback(i, j - 1):
                    alignments.append((a1 + "-", a2 + seq2[j - 1]))
        
        # If no valid moves were found, return the base case
        if not alignments:
            return [("", "")]
            
        return alignments

    # Collect all alignments from maximum scoring positions
    all_alignments = set()
    for i, j in max_positions:
        for aligned_seq1, aligned_seq2 in traceback(i, j):
            all_alignments.add((aligned_seq1, aligned_seq2))
    
    return list(all_alignments), max_score

### Step 4C: Running the Smith–Waterman Algorithm

Now, we test our function using the query and target sequences from the previous steps.

In [10]:
# Call the new smith_waterman function.
alignments, alignment_score = smith_waterman(QUERY, TARGET)

print("=== Smith-Waterman Local Alignment ===\n")
print(f"Alignment Score: {alignment_score}\n")

# Iterate through all optimal alignments.
for idx, (aligned1, aligned2) in enumerate(alignments, start=1):
    print(f"Alignment {idx}:")
    print(f"  Query Alignment:  {aligned1}")
    print(f"  Target Alignment: {aligned2}")
    print()


=== Smith-Waterman Local Alignment ===

Alignment Score: 78

Alignment 1:
  Query Alignment:  TGC-TAGCTAGCTACGATCGATCG-ATCTAGCTAGCTAGCT-A-CGATCGATCG
  Target Alignment: TACGTAGCTAGCT--GATCGATCGTAGCTAGCTAGCTAGCTGATCGATCGATCG

Alignment 2:
  Query Alignment:  ATGCTAGCTAGCTACGATCGATCG-ATCTAGCTAGCTAGCT-A-CGATCGATCG
  Target Alignment: ACG-TAGCTAGCT--GATCGATCGTAGCTAGCTAGCTAGCTGATCGATCGATCG



Each alignment consists of:

- A segment of the query sequence that aligns optimally with the target.
- The highest scoring local match found between the two sequences.


Discussion Points:

1. How does the algorithm decide where to start and stop the alignment?
2. What happens if a target sequence contains mutations?
3. How does the alignment score reflect biological significance?
